In [1]:
import os, sys, os, threading, re

# 1) Tell TF to hide INFO logs, but keep WARNING+ERROR
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 2) Build a small stderr filter that drops only known spam lines
patterns = [
    r'numa_node',                             # NUMA spam
    r'Unable to register cuFFT factory',      # duplicate cuFFT
    r'Unable to register cuDNN factory',      # duplicate cuDNN
    r'Unable to register cuBLAS factory',     # duplicate cuBLAS
    r'gpu_timer\.cc:114',                     # "Skipping the delay kernel"
    r"\+ptx85' is not a recognized feature",  # PTX feature noise
    r"Your kernel may have been built without NUMA support.",  # PTX feature noise
]

compiled = [re.compile(p) for p in patterns]

# We’ll redirect C++ stderr (fd 2) into a pipe, then read/filter in Python
r_fd, w_fd = os.pipe()
orig_stderr_fd = os.dup(2)        # save real stderr

# Point fd 2 (stderr) to the write end of our pipe
os.dup2(w_fd, 2)

def stderr_filter_thread():
    # read from pipe (r_fd), write filtered lines to original stderr (orig_stderr_fd)
    with os.fdopen(r_fd, 'r') as stream, os.fdopen(orig_stderr_fd, 'w') as real_stderr:
        for line in stream:
            if any(p.search(line) for p in compiled):
                # drop known-noise lines
                continue
            real_stderr.write(line)
            real_stderr.flush()

threading.Thread(target=stderr_filter_thread, daemon=True).start()

print("TF log filter installed: INFO hidden, noisy C++ spam filtered, real warnings/errors will still show.")


TF log filter installed: INFO hidden, noisy C++ spam filtered, real warnings/errors will still show.


In [2]:
import tensorflow as tf
tf.config.optimizer.set_jit(False)
import numpy as np

In [3]:
print(tf.__version__)

2.17.0


In [4]:
# tf.constant()

In [5]:
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [6]:
scalar.ndim

0

In [7]:
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [8]:
vector.ndim

1

In [9]:
matrix = tf.constant([[10, 7, 5],
                     [5, 7, 10]])
matrix

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[10,  7,  5],
       [ 5,  7, 10]], dtype=int32)>

In [10]:
matrix.ndim

2

In [11]:
matrix_2 = tf.constant([[10., 7.],
                        [3, 8],
                        [7, 8]],
                       dtype=tf.float16)
matrix_2

2025-12-01 01:15:11.980921: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  8.],
       [ 7.,  8.]], dtype=float16)>

In [12]:
matrix_2.ndim

2

In [13]:
tensor = tf.constant([[[1, 2, 3],
                       [3, 4, 0]],
                      [[5, 5, 1],
                       [8, 1, 7]],
                      [[11, 5, 6],
                       [6, 3, 5]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 3,  4,  0]],

       [[ 5,  5,  1],
        [ 8,  1,  7]],

       [[11,  5,  6],
        [ 6,  3,  5]]], dtype=int32)>

In [14]:
tensor.ndim

3

In [15]:
# Definitions
"""
scalar: is a single number
vector: is a number with direction
matrix: a 2-dimensional array of numbers
tensor: an n-dimensional array of numbers
"""

# Explanation for random_1 != random_2 in cell `EevbQeClBRlI`:
# The reason random_1 and random_2 are not equal, despite both `tf.random.Generator.from_seed(42)` calls, is that `from_seed()` creates a *new and independent* random number generator instance each time it's invoked.
# Even though both generators are initialized with the same seed (42), they are distinct objects. When you call `tf.random.normal` on these separate instances, they produce different random numbers because they are not sharing the same internal state.

# To get identical random tensors, you need to:
# 1. Create a single random number generator instance.
# 2. Use that *same* instance to generate all the tensors you want to be identical.

# Example of how to get identical random tensors:

# Create one generator instance
seeded_generator = tf.random.Generator.from_seed(42)

# Use the same generator to create two tensors
random_a = seeded_generator.normal(shape=(3, 2))
random_b = seeded_generator.normal(shape=(3, 2))

print(f"random_a: {random_a}\n")
print(f"random_b: {random_b}\n")
print(f"Are random_a and random_b equal? {random_a == random_b}")

# If you want two *different* sets of identical tensors, you would re-seed or create a new generator:

# New generator for a different pair, but still identical within the pair
seeded_generator_2 = tf.random.Generator.from_seed(42)
random_c = seeded_generator_2.normal(shape=(3, 2))
seeded_generator_3 = tf.random.Generator.from_seed(42)
random_d = seeded_generator_3.normal(shape=(3, 2))

print(f"\nAre random_c and random_d equal (from separate generators seeded same)? {random_c == random_d}")

# The key is that in the original cell, `random_1` was created by one generator, and `random_2` was created by a *completely new* generator instance, even if both were seeded with 42. They do not share the progression of random numbers.

random_a: [[-0.7565803  -0.06854702]
 [ 0.07595026 -1.2573844 ]
 [-0.23193765 -1.8107855 ]]

random_b: [[ 0.17522676  0.71105534]
 [ 0.54882437  0.14896016]
 [-0.54757965  0.61634356]]

Are random_a and random_b equal? [[False False]
 [False False]
 [False False]]

Are random_c and random_d equal (from separate generators seeded same)? [[ True  True]
 [ True  True]
 [ True  True]]


In [16]:
# tf.Variable()

changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [17]:
changeable_tensor[0].assign(7)
changeable_tensor

# unchangeable_tensor[0].assign(7)

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [18]:
random_1 = tf.random.Generator.from_seed(42)
random_1 = random_1.normal(shape=(3, 2))

random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))

random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193765, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [19]:
# Shuffle the order of elements in a tensor

not_shuffled = tf.constant([
    [10, 7],
    [3, 4],
    [2, 5]
])
not_shuffled.ndim

2

In [20]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [21]:
tf.random.set_seed(42) # Global level random seed
tf.random.shuffle(not_shuffled, seed=42) # Operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [22]:
# How to set random seed on tensorflow

In [23]:
# Other ways of generating tensors

In [24]:
random_3 = tf.constant(np.random.randint(0, 10, size=(3, 2)))
random_3

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[7, 4],
       [9, 3],
       [4, 1]])>

In [25]:
# tf.ones()
tf.ones([3, 4], tf.int32)

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [26]:
#tf.zeros()
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [27]:
# Turn numpy arrays into tensors
"""
Main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU or TPU (for faster numerical processing)
"""

'\nMain difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU or TPU (for faster numerical processing)\n'

In [28]:
numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [29]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A, shape=(3, 8))
C = tf.constant(numpy_A)
A, B, C

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [30]:
# Tensor Attributes
"""
shape --> tensot.shape
rank --> tensor.ndim
axis or dimension tensor[0], tensor[:3]
size --> tf.size(tensor)
"""

'\nshape --> tensot.shape\nrank --> tensor.ndim\naxis or dimension tensor[0], tensor[:3]\nsize --> tf.size(tensor)\n'

In [31]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2, 3, 4, 5])
rank_4_tensor, rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
 array([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]],
 
 
        [[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], dtype=float32)>,
 TensorShape([2, 3, 4, 5]),
 4,
 <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [32]:
rank_4_tensor[0][1][2][4]

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [33]:
# Get various attributes of our tensors

print(f"Datatype of every element: {rank_4_tensor.dtype}")
print(f"Number of dimensions (rank): {rank_4_tensor.ndim}")
print(f"Shape of tensor: {rank_4_tensor.shape}")
print(f"Elements along the 0 axis: {rank_4_tensor.shape[0]}")
print(f"Elements along the last axis: {rank_4_tensor.shape[-1]}")
print(f"Total number of elements in our tensor: {tf.size(rank_4_tensor).numpy()}")

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: 120


In [34]:
# Indexing tensors

In [35]:
# How to get first two elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [36]:
# Get the first element of each dimension from each index excpet the final one
rank_4_tensor[:1, :1, :1, :], rank_4_tensor[:1, :1, :1]

(<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>)

In [37]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [38]:
rank_4_tensor[:1, :, :1, :1]

<tf.Tensor: shape=(1, 3, 1, 1), dtype=float32, numpy=
array([[[[0.]],

        [[0.]],

        [[0.]]]], dtype=float32)>

In [39]:
rank_4_tensor[:, :1, :1, :1]

<tf.Tensor: shape=(2, 1, 1, 1), dtype=float32, numpy=
array([[[[0.]]],


       [[[0.]]]], dtype=float32)>

In [40]:
rank_2_tensor = tf.constant(np.random.randint(1, 20, size=(3, 4)))
rank_2_tensor, rank_2_tensor.shape, rank_2_tensor.ndim, tf.size(rank_2_tensor).numpy()

(<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
 array([[16,  7, 10, 12],
        [ 4, 10, 12, 16],
        [14,  3, 10,  7]])>,
 TensorShape([3, 4]),
 2,
 12)

In [41]:
# How to get last 2 elements in each row
rank_2_tensor[:, -1:]

<tf.Tensor: shape=(3, 1), dtype=int64, numpy=
array([[12],
       [16],
       [ 7]])>

In [42]:
# How to get last 2 elements in each row
rank_2_tensor[:, -1]

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([12, 16,  7])>

In [43]:
# Adding an extra dimension to tensor # tf.newaxis

rank_3_tensor = rank_2_tensor[..., tf.newaxis] # ... means [:, :, tf.newaxis] -> Add all previous axis then add a new axis
rank_3_tensor


<tf.Tensor: shape=(3, 4, 1), dtype=int64, numpy=
array([[[16],
        [ 7],
        [10],
        [12]],

       [[ 4],
        [10],
        [12],
        [16]],

       [[14],
        [ 3],
        [10],
        [ 7]]])>

In [44]:
new_rank_4_tensor = rank_3_tensor[..., tf.newaxis]
new_rank_4_tensor

<tf.Tensor: shape=(3, 4, 1, 1), dtype=int64, numpy=
array([[[[16]],

        [[ 7]],

        [[10]],

        [[12]]],


       [[[ 4]],

        [[10]],

        [[12]],

        [[16]]],


       [[[14]],

        [[ 3]],

        [[10]],

        [[ 7]]]])>

In [45]:
# Alternative to .newaxis  --> tf.expand_dims(rank_2_tensor, axis=-1)
tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(3, 4, 1), dtype=int64, numpy=
array([[[16],
        [ 7],
        [10],
        [12]],

       [[ 4],
        [10],
        [12],
        [16]],

       [[14],
        [ 3],
        [10],
        [ 7]]])>

In [46]:
tf.expand_dims(rank_2_tensor, axis=0)

<tf.Tensor: shape=(1, 3, 4), dtype=int64, numpy=
array([[[16,  7, 10, 12],
        [ 4, 10, 12, 16],
        [14,  3, 10,  7]]])>

In [47]:
tf.expand_dims(rank_2_tensor, axis=1)

<tf.Tensor: shape=(3, 1, 4), dtype=int64, numpy=
array([[[16,  7, 10, 12]],

       [[ 4, 10, 12, 16]],

       [[14,  3, 10,  7]]])>

In [48]:
## Manipulating tensors (tensors operations)
"""
Basic operations: +, -, *, / or equivalent tf operations tf.add, tf.subtract, tf.multiply, tf.divide --> use tf operations to speed up execution
"""

'\nBasic operations: +, -, *, / or equivalent tf operations tf.add, tf.subtract, tf.multiply, tf.divide --> use tf operations to speed up execution\n'

In [49]:
np.random.seed(42)
tensor = tf.constant(np.random.randint(1, 20, size=(3, 4)))
tensor

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 7, 15, 11,  8],
       [ 7, 19, 11, 11],
       [ 4,  8,  3,  2]])>

In [50]:
tensor + 10

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[17, 25, 21, 18],
       [17, 29, 21, 21],
       [14, 18, 13, 12]])>

In [51]:
tensor * 5

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[35, 75, 55, 40],
       [35, 95, 55, 55],
       [20, 40, 15, 10]])>

In [52]:
tensor / 10

<tf.Tensor: shape=(3, 4), dtype=float64, numpy=
array([[0.7, 1.5, 1.1, 0.8],
       [0.7, 1.9, 1.1, 1.1],
       [0.4, 0.8, 0.3, 0.2]])>

In [53]:
tensor - 10

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[-3,  5,  1, -2],
       [-3,  9,  1,  1],
       [-6, -2, -7, -8]])>

In [54]:
tf.add(tensor, 10)

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[17, 25, 21, 18],
       [17, 29, 21, 21],
       [14, 18, 13, 12]])>

In [55]:
tf.multiply(tensor, 10)

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 70, 150, 110,  80],
       [ 70, 190, 110, 110],
       [ 40,  80,  30,  20]])>

In [56]:
tf.subtract(tensor, 10)

<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[-3,  5,  1, -2],
       [-3,  9,  1,  1],
       [-6, -2, -7, -8]])>

In [57]:
tf.divide(tensor, 10)

<tf.Tensor: shape=(3, 4), dtype=float64, numpy=
array([[0.7, 1.5, 1.1, 0.8],
       [0.7, 1.9, 1.1, 1.1],
       [0.4, 0.8, 0.3, 0.2]])>

**Matrix Multiplication**
One of the most common operations is matrix multipliction

There are two rules
1. Inner of dimension sizes must match
2. Output is outer dimensions sizes

In [58]:
A = tf.constant(np.random.randint(-10, 10, size=(3, 2)))
B = tf.constant(np.random.randint(1, 10, size=(2, 3)))

A, B

(<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
 array([[  1,  -5],
        [ -9, -10],
        [  1,   1]])>,
 <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[9, 1, 3],
        [7, 4, 9]])>)

In [59]:
tf.matmul(A, B)

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[ -26,  -19,  -42],
       [-151,  -49, -117],
       [  16,    5,   12]])>

In [60]:
tf.matmul(B, A)

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[  3, -52],
       [-20, -66]])>

In [61]:
A * A

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[  1,  25],
       [ 81, 100],
       [  1,   1]])>

In [62]:
A @ B

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[ -26,  -19,  -42],
       [-151,  -49, -117],
       [  16,    5,   12]])>

In [63]:
A.shape, B.shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [64]:
tf.reshape(A, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[  1,  -5,  -9],
       [-10,   1,   1]])>

In [65]:
tf.matmul(A, tf.reshape(A, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[ 51, -10, -14],
       [ 91,  35,  71],
       [ -9,  -4,  -8]])>

In [66]:
tf.matmul(tf.reshape(A, shape=(2, 3)), A)

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[ 37,  36],
       [-18,  41]])>

In [67]:
# We can reshape with transpose ve reshape
"""
tf.transpose(A). --> rows will be columns and columns will be rows
tf.reshape(A, shape=(2, 3)) --> Element order does not change.
"""

'\ntf.transpose(A). --> rows will be columns and columns will be rows\ntf.reshape(A, shape=(2, 3)) --> Element order does not change.\n'

In [68]:
A, tf.transpose(A), tf.reshape(A, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
 array([[  1,  -5],
        [ -9, -10],
        [  1,   1]])>,
 <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[  1,  -9,   1],
        [ -5, -10,   1]])>,
 <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[  1,  -5,  -9],
        [-10,   1,   1]])>)

In [69]:
# Matrix multiplication with tf.transpose vs tf.reshape
tf.matmul(tf.transpose(A), A), tf.matmul(tf.reshape(A, shape=(2, 3)), A)

(<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
 array([[ 83,  86],
        [ 86, 126]])>,
 <tf.Tensor: shape=(2, 2), dtype=int64, numpy=
 array([[ 37,  36],
        [-18,  41]])>)

In [70]:
# we can do matrix multiplication in two ways
"""
1. tf.matmul()
2. tf.tensordot()
3. `@` for python operation
"""

'\n1. tf.matmul()\n2. tf.tensordot()\n3. `@` for python operation\n'

In [71]:
np.random.seed(42)
X = tf.constant(np.random.randint(1, 20, size=(3, 2)))
Y = tf.constant(np.random.randint(1, 20, size=(3, 2)))

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
 array([[ 7, 15],
        [11,  8],
        [ 7, 19]])>,
 <tf.Tensor: shape=(3, 2), dtype=int64, numpy=
 array([[11, 11],
        [ 4,  8],
        [ 3,  2]])>)

In [72]:
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[142, 179],
       [254, 267]])>

In [73]:
tf.tensordot(X, tf.transpose(Y), axes=1)

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[242, 148,  51],
       [209, 108,  49],
       [286, 180,  59]])>

In [74]:
# Perform matrix multiplication

In [75]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int64, numpy=
array([[242, 148,  51],
       [209, 108,  49],
       [286, 180,  59]])>

In [76]:
Y, tf.transpose(Y), tf.reshape(Y, shape=(2, 3))

(<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
 array([[11, 11],
        [ 4,  8],
        [ 3,  2]])>,
 <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[11,  4,  3],
        [11,  8,  2]])>,
 <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[11, 11,  4],
        [ 8,  3,  2]])>)

In [77]:
# Changing data type of tensor
# change from float32 to float16 --> reduced precision

X = tf.constant([1.7, 7.4])
X, X.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [78]:
Y = tf.cast(X, dtype=tf.float16)
X, X.dtype, Y, Y.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32,
 <tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [79]:
A = tf.constant([1, 2, 3])

B = tf.cast(A, dtype=tf.float32)
A, A.dtype, B, B.dtype

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>,
 tf.int32,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>,
 tf.float32)

In [80]:
## Aggregating Tensors : Condensing them from multiple values down to a smaller amount of values

# Getting the absolute values. - tf.abs()

X = tf.constant([-1, -2, -3])
tf.abs(X)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [81]:
# Get minimum
# Get Maximum
# Get Mean
# Get Sum
# Get std
# Get Variance


np.random.seed(42)
tensor = tf.constant(np.random.randint(-10, 100, size=(3, 25)))
tensor = tf.cast(tensor, dtype=tf.float32)


print(f"{tensor}\n")

tf.minimum(tensor, tensor)
print(f"Min : {tf.reduce_min(tensor)}")
print(f"Max : {tf.reduce_max(tensor)}")
print(f"Mean : {tf.reduce_mean(tensor)}")
print("Sum:", tf.reduce_sum(tensor).numpy())
print("Mean:", tf.reduce_mean(tensor).numpy())
print("Variance:", tf.math.reduce_variance(tensor).numpy())
print("Std Dev:", tf.math.reduce_std(tensor).numpy())



[[92. 41. 82.  4. 96. 61. 50. 10. 92. 72. 76. 64. 64. 77. 89. 93. 13. -8.
  11. 42. -9. 77. 97. 19. 27.]
 [-9. 53. 49. 10. 22. 65. 47. 11. 97. 78. 38. 80. 48. 31. 81. 49. 69.  4.
  51. 51. 36. 51. 40. 97. 44.]
 [53. -8. 90. 40. -4. 10. 62. 28.  7. -7. 78. 49.  3. -2. 79. 42. -9. 73.
  81. 49. 60. 33. -3. 36. 24.]]

Min : -9.0
Max : 97.0
Mean : 45.186668395996094
Sum: 3389.0
Mean: 45.18667
Variance: 1036.0184
Std Dev: 32.18724


In [82]:
# One hot encoding with tensors
some_list = [0, 1, 2, 3] # class labels
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [83]:
np.random.seed(42)
H = tf.constant(np.random.randint(0, 4, size=(2, 6)))
H_encoded=tf.one_hot(H, depth=4)
H_custom_encoded=tf.one_hot(H, depth=4, on_value='O', off_value='X')
H, H_encoded, H_custom_encoded

(<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
 array([[2, 3, 0, 2, 2, 3],
        [0, 0, 2, 1, 2, 2]])>,
 <tf.Tensor: shape=(2, 6, 4), dtype=float32, numpy=
 array([[[0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],
 
        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 6, 4), dtype=string, numpy=
 array([[[b'X', b'X', b'O', b'X'],
         [b'X', b'X', b'X', b'O'],
         [b'O', b'X', b'X', b'X'],
         [b'X', b'X', b'O', b'X'],
         [b'X', b'X', b'O', b'X'],
         [b'X', b'X', b'X', b'O']],
 
        [[b'O', b'X', b'X', b'X'],
         [b'O', b'X', b'X', b'X'],
         [b'X', b'X', b'O', b'X'],
         [b'X', b'O', b'X', b'X'],
         [b'X', b'X', b'O', b'X'],
         [b'X', b'X', b'O', b'X']]], dtype=object)>)

In [84]:
H

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[2, 3, 0, 2, 2, 3],
       [0, 0, 2, 1, 2, 2]])>

In [85]:
# Squeezing a tensor (removing all single dimensions)
I = tf.range(1, 10)
tf.square(I), tf.sqrt(tf.cast(I, dtype=tf.float32))

(<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>,
 <tf.Tensor: shape=(9,), dtype=float32, numpy=
 array([1.       , 1.4142135, 1.7320508, 2.       , 2.2360678, 2.4494896,
        2.6457512, 2.828427 , 3.       ], dtype=float32)>)

In [86]:
tf.math.log(tf.cast(I, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

In [87]:
tf.math.negative(I)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([-1, -2, -3, -4, -5, -6, -7, -8, -9], dtype=int32)>

In [88]:
tf.math.reciprocal(tf.cast(I, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.        , 0.5       , 0.33333334, 0.25      , 0.2       ,
       0.16666667, 0.14285715, 0.125     , 0.11111111], dtype=float32)>

In [89]:
J = tf.random.uniform(shape=(10,), minval=1, maxval=10, dtype=tf.int32)
K = tf.random.uniform(shape=(10,), minval=5, maxval=15, dtype=tf.int32)

J, K, tf.math.squared_difference(J, K)

(<tf.Tensor: shape=(10,), dtype=int32, numpy=array([8, 6, 9, 4, 8, 3, 8, 8, 8, 7], dtype=int32)>,
 <tf.Tensor: shape=(10,), dtype=int32, numpy=array([13,  8, 14,  9,  7,  8,  9,  7, 11,  9], dtype=int32)>,
 <tf.Tensor: shape=(10,), dtype=int32, numpy=array([25,  4, 25, 25,  1, 25,  1,  1,  9,  4], dtype=int32)>)

In [90]:
### Tensors and NumPy
# Tensorflow interacts perfectly with Numpy array

In [91]:
K = tf.constant(np.random.randint(1, 10, size=(3, 4)))
K.numpy(), type(K.numpy())

(array([[8, 5, 4, 8],
        [8, 3, 6, 5],
        [2, 8, 6, 2]]),
 numpy.ndarray)

In [92]:
print(K.numpy()[0][2])

4


In [93]:
# Default type of numpy array is float64 whereas tensorflow tensor is float32

numpy_K = tf.constant(np.array([3., 7., 10.]))
tensor_K = tf.constant([3., 7., 10.])

numpy_K.dtype, tensor_K.dtype

(tf.float64, tf.float32)

In [94]:
tf.config.list_physical_devices('GPU'), tf.config.list_physical_devices('CPU'), tf.config.list_physical_devices()

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')],
 [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')],
 [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')])

In [95]:
with tf.device("/GPU:0"):
    a = tf.random.normal((4000, 4000))
    b = tf.random.normal((4000, 4000))
    c = tf.matmul(a, b)
print("OK, matmul shape:", c.shape)


OK, matmul shape: (4000, 4000)


In [96]:
!nvidia-smi

Mon Dec  1 01:15:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.102.01             Driver Version: 581.57         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   43C    P4             15W /   55W |    7533MiB /  12227MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [97]:
import tensorflow as tf
import time

print("TensorFlow version:", tf.__version__)
print("Physical devices:", tf.config.list_physical_devices())

# Tensor sizes for heavy workload
N = 5000

# Warmup
a = tf.random.normal((N, N))
b = tf.random.normal((N, N))
_ = tf.matmul(a, b)

# Compute time test
def benchmark(device):
    with tf.device(device):
        a = tf.random.normal((N, N))
        b = tf.random.normal((N, N))

        start = time.time()
        c = tf.matmul(a, b)
        tf.experimental.numpy.sum(c)  # force computation
        end = time.time()

    return end - start

cpu_time = benchmark("/CPU:0")
print(f"\nCPU time: {cpu_time:.4f} seconds")

if tf.config.list_physical_devices('GPU'):
    gpu_time = benchmark("/GPU:0")
    print(f"GPU time: {gpu_time:.4f} seconds")
else:
    print("\nNo GPU detected.")

TensorFlow version: 2.17.0
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

CPU time: 0.1637 seconds
GPU time: 0.0005 seconds


In [98]:
import tensorflow as tf
import time

def build_model():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(1024, activation="relu", input_shape=(784,)),
        tf.keras.layers.Dense(1024, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0

def train_on(device, epochs=3):
    with tf.device(device):
        model = build_model()
        model.compile(optimizer="adam",
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])
        start = time.time()
        model.fit(x_train, y_train, batch_size=512, epochs=epochs, verbose=0)
        end = time.time()
    return end - start

cpu_time = train_on("/CPU:0")
gpu_time = train_on("/GPU:0")

print(f"CPU train time: {cpu_time:.3f} s")
print(f"GPU train time: {gpu_time:.3f} s")

I0000 00:00:1764551714.199623    1298 service.cc:146] XLA service 0x77457006bc60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764551714.199653    1298 service.cc:154]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1764551714.232508    1298 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1764551719.963476    1301 service.cc:146] XLA service 0x77455c249aa0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764551719.963503    1301 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 5070 Ti Laptop GPU, Compute Capability 12.0


CPU train time: 5.487 s
GPU train time: 1.850 s
